In [319]:
import pandas as pd
import sqlite3

In [320]:
class WeatherDatabase:
    url: str

    def __init__(self, url: str, *args):
        self.url = url
        con = sqlite3.connect(url)
        cur = con.cursor()
        for table in args:
            con.execute(
                f"""
                CREATE DATABASE IF NOT EXISTS Expences
                """)
        cur.close()
        con.commit()
        con.close()

    def call_db(self, query, *args):
        con = sqlite3.connect(self.url)
        cur = con.cursor()
        res = cur.execute(query, args)
        data = res.fetchall()
        cur.close()
        con.commit()
        con.close()
        return data

In [321]:
db = WeatherDatabase('./WeatherGBG.db')

In [322]:
query = """SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_temp
            FROM Temp_Save_raw
            GROUP BY Datum"""


In [323]:
conn = sqlite3.connect('./WeatherGBG.db')

In [324]:
df_avg_temp_daily = pd.read_sql_query(query, conn)

In [325]:
df_avg_temp_daily = df_avg_temp_daily.drop('id', axis=1)

In [326]:
df_avg_temp_daily = df_avg_temp_daily.drop(['Lufttemperatur', 'Tid_UTC'], axis=1)

In [327]:
df_avg_temp_daily

,Datum,Kvalitet,Datum_date,Avg_temp
0,1944-01-01,G,1944-01-01,3.20
1,1944-01-02,G,1944-01-02,3.60
2,1944-01-03,G,1944-01-03,2.80
3,1944-01-04,G,1944-01-04,-4.47
4,1944-01-05,G,1944-01-05,-1.40
...,...,...,...,...
22980,2006-12-01,G,2006-12-01,8.03
22981,2006-12-02,G,2006-12-02,8.81
22982,2006-12-03,G,2006-12-03,8.11
22983,2006-12-04,G,2006-12-04,8.57


In [328]:
# Compare to previous dataframe
pd.read_pickle('../Dataframes/df_daily_temp.pkl')

,Datum,Kvalitet,Snittemperatur,Year
0,1944-01-01,G,3.20,1944-01-01
1,1944-01-02,G,3.60,1944-01-01
2,1944-01-03,G,2.80,1944-01-01
3,1944-01-04,G,-4.47,1944-01-01
4,1944-01-05,G,-1.40,1944-01-01
...,...,...,...,...
29109,2023-10-12,G,12.08,2023-01-01
29110,2023-10-13,G,11.89,2023-01-01
29111,2023-10-14,G,11.54,2023-01-01
29112,2023-10-15,G,10.09,2023-01-01


In [329]:
month_year_query = """SELECT *,
                        ROUND(AVG(Lufttemperatur), 2) AS Avg_temp,
                        SUBSTR(Datum, 1, 4) AS Year,
                        SUBSTR(Datum, 6, 2) AS Month
            FROM Temp_Save_raw
            GROUP BY Datum"""


In [330]:
df = pd.read_sql_query(month_year_query, conn)
df.drop('id', axis=1)

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Datum_date,Avg_temp,Year,Month
0,1944-01-01,07:00:00,1.6,G,1944-01-01,3.20,1944,01
1,1944-01-02,07:00:00,0.6,G,1944-01-02,3.60,1944,01
2,1944-01-03,07:00:00,3.8,G,1944-01-03,2.80,1944,01
3,1944-01-04,07:00:00,-3.4,G,1944-01-04,-4.47,1944,01
4,1944-01-05,07:00:00,-6.8,G,1944-01-05,-1.40,1944,01
...,...,...,...,...,...,...,...,...
22980,2006-12-01,00:00:00,8.3,G,2006-12-01,8.03,2006,12
22981,2006-12-02,00:00:00,9.2,G,2006-12-02,8.81,2006,12
22982,2006-12-03,00:00:00,8.5,G,2006-12-03,8.11,2006,12
22983,2006-12-04,00:00:00,9.3,G,2006-12-04,8.57,2006,12


In [331]:
last_date = df[-1:]['Datum'].values[0]

In [332]:
last_date

'2006-12-05'

In [333]:
month_year_query_gbg = """SELECT *,
                        ROUND(AVG(Lufttemperatur), 2) AS Avg_temp,
                        SUBSTR(Datum, 1, 4) AS Year,
                        SUBSTR(Datum, 6, 2) AS Month
            FROM Temp_GBG_raw
            WHERE Datum > 2006-12-05
            GROUP BY Datum"""

In [334]:
df = pd.read_sql_query(month_year_query_gbg, conn)
df.drop('id', axis=1)

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Datum_date,Avg_temp,Year,Month
0,1961-01-01,06:00:00,0.8,G,1961-01-01,1.07,1961,01
1,1961-01-02,06:00:00,1.8,G,1961-01-02,2.27,1961,01
2,1961-01-03,06:00:00,2.6,G,1961-01-03,2.27,1961,01
3,1961-01-04,06:00:00,2.8,G,1961-01-04,2.53,1961,01
4,1961-01-05,06:00:00,1.8,G,1961-01-05,2.13,1961,01
...,...,...,...,...,...,...,...,...
19909,2023-06-27,00:00:00,16.3,G,2023-06-27,18.18,2023,06
19910,2023-06-28,00:00:00,16.2,G,2023-06-28,19.73,2023,06
19911,2023-06-29,00:00:00,16.8,G,2023-06-29,19.23,2023,06
19912,2023-06-30,00:00:00,19.2,G,2023-06-30,17.40,2023,06


In order sort by dates, the dates must be in date-format
To do this, create a new column that has the dates in a date-format, see below

In [335]:
add_colum = """ALTER TABLE Temp_GBG_raw
                ADD COLUMN Datum_date DATE"""

In [336]:
db.call_db(add_colum)

OperationalError: duplicate column name: Datum_date

In [ ]:
add_dates_to_column = """UPDATE Temp_GBG_raw
                            SET Datum_date = DATE(Datum)"""

In [ ]:
db.call_db(add_dates_to_column)

[]

In [ ]:
# Temp for Säve

add_colum = """ALTER TABLE Temp_Save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum)
add_dates_to_column = """UPDATE Temp_Save_raw
                            SET Datum_date = DATE(Datum)"""
db.call_db(add_dates_to_column)

[]

In [ ]:
# Precipitation for GBG

add_colum_gbg = """ALTER TABLE precipitation_data_GBG_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_gbg)
add_dates_to_column_gbg = """UPDATE precipitation_data_GBG_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_gbg)

[]

In [ ]:
# Precipitation for Säve

add_colum_save = """ALTER TABLE precipitation_data_save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_save)
add_dates_to_column_save = """UPDATE precipitation_data_save_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_save)

[]

Let's make some queries

In [337]:
df1 = pd.read_sql_query('SELECT * FROM Temp_GBG_raw', conn, parse_dates=['Datum_date']).drop('id', axis=1)
df1

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Datum_date
0,1961-01-01,06:00:00,0.8,G,1961-01-01
1,1961-01-01,12:00:00,1.0,G,1961-01-01
2,1961-01-01,18:00:00,1.4,G,1961-01-01
3,1961-01-02,06:00:00,1.8,G,1961-01-02
4,1961-01-02,12:00:00,2.2,G,1961-01-02
...,...,...,...,...,...
308145,2023-07-01,02:00:00,16.1,G,2023-07-01
308146,2023-07-01,03:00:00,16.1,G,2023-07-01
308147,2023-07-01,04:00:00,16.2,G,2023-07-01
308148,2023-07-01,05:00:00,16.3,G,2023-07-01


In [338]:
df1.dtypes

Datum                     object
Tid_UTC                   object
Lufttemperatur           float64
Kvalitet                  object
Datum_date        datetime64[ns]
dtype: object

In [339]:
last_date

'2006-12-05'

In [340]:
# This df only contains values from 2006-12-16, the first day Säve station was not in use.
# Use this to union with df from Säve to create full DF

df_gbg_temp = pd.read_sql("""SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                  FROM Temp_GBG_raw 
                  WHERE Datum_date > '2006-12-05'
                  GROUP BY Datum_date
                  """, conn, parse_dates=['Datum_date']).drop(['Tid_UTC', 'Lufttemperatur', 'id'], axis=1)
df_gbg_temp

,Datum,Kvalitet,Datum_date,Avg_daily_temp
0,2006-12-06,G,2006-12-06,9.29
1,2006-12-07,G,2006-12-07,7.70
2,2006-12-08,G,2006-12-08,8.86
3,2006-12-09,G,2006-12-09,8.58
4,2006-12-10,G,2006-12-10,7.52
...,...,...,...,...
6017,2023-06-27,G,2023-06-27,18.18
6018,2023-06-28,G,2023-06-28,19.73
6019,2023-06-29,G,2023-06-29,19.23
6020,2023-06-30,G,2023-06-30,17.40


In [341]:
df_save_temp = pd.read_sql("""SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                  FROM Temp_save_raw 
                  GROUP BY Datum_date
                  """, conn, parse_dates=['Datum_date']).drop(['Tid_UTC', 'Lufttemperatur', 'id'], axis=1)
df_save_temp

,Datum,Kvalitet,Datum_date,Avg_daily_temp
0,1944-01-01,G,1944-01-01,3.20
1,1944-01-02,G,1944-01-02,3.60
2,1944-01-03,G,1944-01-03,2.80
3,1944-01-04,G,1944-01-04,-4.47
4,1944-01-05,G,1944-01-05,-1.40
...,...,...,...,...
22980,2006-12-01,G,2006-12-01,8.03
22981,2006-12-02,G,2006-12-02,8.81
22982,2006-12-03,G,2006-12-03,8.11
22983,2006-12-04,G,2006-12-04,8.57


In [342]:
df_compiled = pd.read_sql("""SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                  FROM Temp_save_raw 
                  GROUP BY Datum_date
            UNION ALL
                SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                  FROM Temp_GBG_raw 
                  WHERE Datum_date > '2006-12-05'
                  GROUP BY Datum_date
                  """, conn, parse_dates=['Datum_date']).drop(['Tid_UTC', 'Lufttemperatur', 'id'], axis=1)
df_compiled

,Datum,Kvalitet,Datum_date,Avg_daily_temp
0,1944-01-01,G,1944-01-01,3.20
1,1944-01-02,G,1944-01-02,3.60
2,1944-01-03,G,1944-01-03,2.80
3,1944-01-04,G,1944-01-04,-4.47
4,1944-01-05,G,1944-01-05,-1.40
...,...,...,...,...
29002,2023-06-27,G,2023-06-27,18.18
29003,2023-06-28,G,2023-06-28,19.73
29004,2023-06-29,G,2023-06-29,19.23
29005,2023-06-30,G,2023-06-30,17.40


In [343]:
df_compiled.dtypes

Datum                     object
Kvalitet                  object
Datum_date        datetime64[ns]
Avg_daily_temp           float64
dtype: object

In [344]:
# Test on a smaller Union to make sure it works

df_smaller = pd.read_sql("""SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                    FROM Temp_save_raw 
                    WHERE Datum_date < '1944-01-03'
                    GROUP BY Datum_date
            UNION ALL
                SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                  FROM Temp_GBG_raw 
                  WHERE Datum_date > '2023-06-29'
                  GROUP BY Datum_date
                  """, conn, parse_dates=['Datum_date']).drop(['Tid_UTC', 'Lufttemperatur', 'id'], axis=1)
df_smaller

,Datum,Kvalitet,Datum_date,Avg_daily_temp
0,1944-01-01,G,1944-01-01,3.20
1,1944-01-02,G,1944-01-02,3.60
2,2023-06-30,G,2023-06-30,17.40
3,2023-07-01,G,2023-07-01,16.06


Create a column containing boolea value for if the previous day was warmer

In [345]:
df_warmer = pd.read_sql("""WITH Temp_comp AS (
                        SELECT *, LAG(Lufttemperatur) OVER (ORDER BY Datum_date) AS prev_temp
                        FROM Temp_save_raw)
                        SELECT *,
                        CASE
                          WHEN Lufttemperatur > prev_temp THEN 1
                          ELSE 0
                        END AS Warmer
                      FROM Temp_comp;
                  """, conn)
df_warmer

,id,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Datum_date,prev_temp,Warmer
0,1,1944-01-01,07:00:00,1.6,G,1944-01-01,NaN,0
1,2,1944-01-01,13:00:00,3.8,G,1944-01-01,1.6,1
2,3,1944-01-01,18:00:00,4.2,G,1944-01-01,3.8,1
3,4,1944-01-02,07:00:00,0.6,G,1944-01-02,4.2,0
4,5,1944-01-02,13:00:00,3.8,G,1944-01-02,0.6,1
...,...,...,...,...,...,...,...,...
435387,435388,2006-12-05,09:00:00,9.4,G,2006-12-05,8.5,1
435388,435389,2006-12-05,10:00:00,10.1,Y,2006-12-05,9.4,1
435389,435390,2006-12-05,11:00:00,10.8,Y,2006-12-05,10.1,1
435390,435391,2006-12-05,12:00:00,10.9,G,2006-12-05,10.8,1


In [346]:
df_warmer = pd.read_sql("""WITH Temp_comp AS (
                        SELECT *, LAG(Avg_daily_temp) OVER (ORDER BY Datum_date) AS Prev_temp
                        FROM 
                            (SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                            FROM Temp_save_raw 
                            GROUP BY Datum_date))
                        SELECT *,
                        CASE WHEN Avg_daily_temp > Prev_temp THEN 1
                        ELSE 0
                        END AS Warmer
                        FROM Temp_comp;
                        """, conn).drop(columns=['Tid_UTC', 'id'])
df_warmer

,Datum,Lufttemperatur,Kvalitet,Datum_date,Avg_daily_temp,Prev_temp,Warmer
0,1944-01-01,1.6,G,1944-01-01,3.20,NaN,0
1,1944-01-02,0.6,G,1944-01-02,3.60,3.20,1
2,1944-01-03,3.8,G,1944-01-03,2.80,3.60,0
3,1944-01-04,-3.4,G,1944-01-04,-4.47,2.80,0
4,1944-01-05,-6.8,G,1944-01-05,-1.40,-4.47,1
...,...,...,...,...,...,...,...
22980,2006-12-01,8.3,G,2006-12-01,8.03,9.98,0
22981,2006-12-02,9.2,G,2006-12-02,8.81,8.03,1
22982,2006-12-03,8.5,G,2006-12-03,8.11,8.81,0
22983,2006-12-04,9.3,G,2006-12-04,8.57,8.11,1


Try above with Union

In [387]:
df_warmer_union = pd.read_sql("""WITH Temp_comp AS (
                        SELECT *, LAG(Avg_daily_temp) OVER (ORDER BY Datum_date) AS Prev_temp
                        FROM 
                            (SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                            FROM Temp_save_raw 
                            GROUP BY Datum_date
                                UNION ALL
                            SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_daily_temp 
                            FROM Temp_GBG_raw 
                            WHERE Datum_date > '2006-12-05'
                            GROUP BY Datum_date))
                        SELECT *,
                        CASE WHEN Avg_daily_temp > Prev_temp THEN 1
                        ELSE 0
                        END AS Warmer
                        FROM Temp_comp;
                        """, conn).drop(columns=['Tid_UTC', 'id']).drop(columns=['Prev_temp'])
df_warmer_union

,Datum,Lufttemperatur,Kvalitet,Datum_date,Avg_daily_temp,Warmer
0,1944-01-01,1.6,G,1944-01-01,3.20,0
1,1944-01-02,0.6,G,1944-01-02,3.60,1
2,1944-01-03,3.8,G,1944-01-03,2.80,0
3,1944-01-04,-3.4,G,1944-01-04,-4.47,0
4,1944-01-05,-6.8,G,1944-01-05,-1.40,1
...,...,...,...,...,...,...
29002,2023-06-27,16.3,G,2023-06-27,18.18,0
29003,2023-06-28,16.2,G,2023-06-28,19.73,1
29004,2023-06-29,16.8,G,2023-06-29,19.23,0
29005,2023-06-30,19.2,G,2023-06-30,17.40,0


In [ ]:
def try_as_func(table, date):
    cursor = conn.cursor()
    cursor.execute(f'SELECT * FROM {table} WHERE Datum_date >= ?', (date,))
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return pd.DataFrame(results)

df = try_as_func('Temp_GBG_raw', '2002-01-01')

In [ ]:
def try_pd_read_sql(table, date):
    conn = sqlite3.connect('./WeatherGBG.db')
    query = f"""SELECT * FROM {table} WHERE Datum_date >= ?"""
    res = pd.read_sql(query, conn, params=(date,)).drop(columns=['id'])
    conn.close()
    return res

In [ ]:
df = try_pd_read_sql('Temp_GBG_raw', '2002-01-01')

In [ ]:
df

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Datum_date
0,2002-01-01,00:00:00,-16.6,G,2002-01-01
1,2002-01-01,01:00:00,-16.7,Y,2002-01-01
2,2002-01-01,02:00:00,-16.4,Y,2002-01-01
3,2002-01-01,03:00:00,-13.7,G,2002-01-01
4,2002-01-01,04:00:00,-9.8,Y,2002-01-01
...,...,...,...,...,...
184950,2023-07-01,02:00:00,16.1,G,2023-07-01
184951,2023-07-01,03:00:00,16.1,G,2023-07-01
184952,2023-07-01,04:00:00,16.2,G,2023-07-01
184953,2023-07-01,05:00:00,16.3,G,2023-07-01


In [ ]:
# for ind, val in df_warmer_union[2:100].iterrows():
#     if df_warmer_union['Lufttemperatur'][ind] > df_warmer_union['Lufttemperatur'][ind-1]:
#         df_warmer_union['Varmare'][ind] = df_warmer_union['Lufttemperatur'][ind-1]
#     elif KeyError:
#         pass
#     else:
#         df_warmer_union['Varmare'] = 0


In [388]:
df_warmer_union

,Datum,Lufttemperatur,Kvalitet,Datum_date,Avg_daily_temp,Warmer
0,1944-01-01,1.6,G,1944-01-01,3.20,0
1,1944-01-02,0.6,G,1944-01-02,3.60,1
2,1944-01-03,3.8,G,1944-01-03,2.80,0
3,1944-01-04,-3.4,G,1944-01-04,-4.47,0
4,1944-01-05,-6.8,G,1944-01-05,-1.40,1
...,...,...,...,...,...,...
29002,2023-06-27,16.3,G,2023-06-27,18.18,0
29003,2023-06-28,16.2,G,2023-06-28,19.73,1
29004,2023-06-29,16.8,G,2023-06-29,19.23,0
29005,2023-06-30,19.2,G,2023-06-30,17.40,0


In [481]:
ind = 1
df_warmer_union[ind-1:ind]['Avg_daily_temp'][ind-1]

3.2

In [487]:
df_warmer_union['Steg_tempern?'] = 0
counter = 0


for ind, val in df_warmer_union[1:1500].iterrows():
    temp_prev = df_warmer_union[ind-1:ind]['Avg_daily_temp'][ind-1]
    # print(temp_prev)
    # break
    # break
    # temp_prev = str(temp_prev)
    # temp_prev = temp_prev[-41:-37]
    # temp_prev = float(temp_prev)
    # print(temp_prev)
    # break
    # temp_prev = temp_prev[5:8]
    # print(temp_prev)
    # temp_prev = float(temp_prev)
    # print(temp_prev)
 
    # if ValueError:
    #     print(temp_prev)
    # print(type(df_warmer_union['Steg_tempern?'][1]))
    if val['Avg_daily_temp'] > temp_prev:
        counter += 1
    else:
        counter = 0
    if counter == 5:
        df_warmer_union.loc[ind, 'Steg_tempern?'] = 1
    else:
        df_warmer_union.loc[ind, 'Steg_tempern?'] = 0


In [ ]:
df_warmer_union[100:150]

Skapa kolumn som markerar om det varit över 10 grader i 5 dagar

In [494]:
df_warmer_union['Över_10_grader_i_5_dagar'] = 0
counter = 0
for ind, val in df_warmer_union[1:1500].iterrows():
    temp_prev = df_warmer_union[ind-1:ind]['Avg_daily_temp'][ind-1]
    if val['Avg_daily_temp'] > 10:
        counter += 1
    else:
        counter = 0
    if counter >= 5:
        df_warmer_union.loc[ind, 'Över_10_grader_i_5_dagar'] = 1
    else:
        df_warmer_union.loc[ind, 'Steg_tempern?'] = 0

In [513]:
df_warmer_union[265:300].drop(columns=['Warmer', 'Steg_tempern?'])

,Datum,Lufttemperatur,Kvalitet,Datum_date,Avg_daily_temp,Över_10_grader_i_5_dagar
265,1944-09-22,12.0,G,1944-09-22,13.80,1
266,1944-09-23,10.8,G,1944-09-23,14.80,1
267,1944-09-24,11.2,G,1944-09-24,12.00,1
268,1944-09-25,8.6,G,1944-09-25,9.23,0
269,1944-09-26,10.2,G,1944-09-26,11.87,0
270,1944-09-27,10.2,G,1944-09-27,10.83,0
271,1944-09-28,10.6,G,1944-09-28,11.47,0
272,1944-09-29,10.2,G,1944-09-29,11.33,0
273,1944-09-30,12.0,G,1944-09-30,11.60,1
274,1944-10-01,10.6,G,1944-10-01,10.67,1
